In [ ]:
# pip install mysql-connector-python
# pip install PyMySQL

## Keep N of File

In [ ]:
import os

def keep_last_n_files(folder_path, n):
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
             if os.path.isfile(os.path.join(folder_path, f))]
    files.sort(key=os.path.getmtime, reverse=True)
    for file_to_delete in files[n:]:
        try:
            os.remove(file_to_delete)
            print(f"Deleted: {file_to_delete}")
        except Exception as e:
            print(f"Failed to delete {file_to_delete}: {e}")

if __name__ == "__main__":
    folder = "path/to/your/folder"  
    max_files = 5                  

    keep_last_n_files(folder, max_files)


## Full Backup

In [ ]:
import subprocess
import json
import os
import datetime
import re

CONFIG_DIR = './config'
CONFIG_FILE = os.path.join(CONFIG_DIR, 'last_full_backup.json')
DEFAULT_BACKUP_DIR = '../backup'


def full_backup(host, user, password, db_name):
    """
    Performs a full backup of a specified database using mysqldump.
    It captures the binary log position at the time of the backup, which is
    essential for subsequent differential backups.
    """
    print("--- Starting Full Backup ---")
    
    # Ensure the backup and config directories exist
    full_backup_dir = os.path.join(DEFAULT_BACKUP_DIR, 'full')
    os.makedirs(full_backup_dir, exist_ok=True)
    os.makedirs(CONFIG_DIR, exist_ok=True)

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_file = os.path.join(full_backup_dir, f"full_backup_{db_name}_{timestamp}.sql")

    # Command to create a full backup and include master data
    command = [
        "mysqldump",
        f"--host={host}",
        f"--user={user}",
        f"--password={password}",
        "--single-transaction",  # Ensures a consistent snapshot
        "--master-data=2",       # Includes binlog file and position in the dump
        db_name
    ]
    
    print(f"Running mysqldump for database '{db_name}'...")
    
    try:
        # Redirect stdout directly to the output file
        with open(output_file, 'w', encoding='utf-8') as f:
            result = subprocess.run(command, stdout=f, stderr=subprocess.PIPE, check=True)

        print(f"Full backup successful. File saved as {output_file}")

        # Extract binlog info from the created dump file
        binlog_info = {}
        with open(output_file, 'r', encoding='utf-8') as f:
            for line in f:
                # CORRECTED: Regex now handles the optional '-- ' comment prefix
                match = re.search(r"--\s*CHANGE MASTER TO MASTER_LOG_FILE='(.+?)',\s*MASTER_LOG_POS=(\d+);", line)
                if match:
                    binlog_info['binlog_file'] = match.group(1)
                    binlog_info['binlog_pos'] = int(match.group(2))
                    break
        
        if binlog_info:
            with open(CONFIG_FILE, 'w', encoding='utf-8') as jf:
                json.dump(binlog_info, jf, indent=4)
            print(f"Binlog info saved to {CONFIG_FILE}: {binlog_info}")
        else:
            print("Warning: Could not find binlog info in the backup file.")

    except subprocess.CalledProcessError as e:
        print(f"Error during full backup: {e.stderr.decode('utf-8')}")
    except FileNotFoundError:
        print("Error: 'mysqldump' command not found. Is the MySQL client installed and in your system's PATH?")

full_backup(
    host="localhost",
    user="root",
    password="1234",
    db_name="school_db" 
)

## Differential Backup

In [ ]:
import subprocess
import json
import os
import datetime

DEFAULT_BACKUP_DIR = '../backup'
CONFIG_DIR = './config'
CONFIG_FILE = os.path.join(CONFIG_DIR, 'last_full_backup.json')

def differential_backup(host, user, password, databases):
    """
    Creates a differential backup using mysqlbinlog.
    It reads the starting binlog file and position from the config created
    by the last full backup and captures all changes since that point.
    """
    print("\n--- Starting Differential Backup ---")

    if not os.path.exists(CONFIG_FILE):
        print(f"Error: Config file not found at {CONFIG_FILE}. Please run a full backup first.")
        return

    with open(CONFIG_FILE, 'r') as f:
        try:
            binlog_info = json.load(f)
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {CONFIG_FILE}.")
            return

    binlog_file = binlog_info.get('binlog_file')
    start_position = binlog_info.get('binlog_pos')

    if not binlog_file or not start_position:
        print("Error: Invalid binlog info in config file. It should contain 'binlog_file' and 'binlog_pos'.")
        return

    diff_backup_dir = os.path.join(DEFAULT_BACKUP_DIR, 'differential')
    os.makedirs(diff_backup_dir, exist_ok=True)

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    db_str = "_".join(databases)
    output_file = os.path.join(diff_backup_dir, f"diff_backup_{db_str}_{timestamp}.sql")

    command = [
        "mysqlbinlog",
        f"--start-position={start_position}",
        "--read-from-remote-server",
        f"--host={host}",
        f"--user={user}",
        f"--password={password}",
    ]

    for db in databases:
        command.append(f"--database={db}")
    
    command.append(binlog_file)

    print(f"Downloading and filtering binlog '{binlog_file}' from position {start_position}...")
    print("Filtering for databases:", ", ".join(databases))

    try:
        with open(output_file, "w", encoding='utf-8') as f:
            result = subprocess.run(command, stdout=f, stderr=subprocess.PIPE, check=True)
        
        print(f"Differential backup successful. File saved as {output_file}.")

    except subprocess.CalledProcessError as e:
        stderr_output = e.stderr.decode('utf-8')
        if "bad magic number" in stderr_output:
             print(f"Error: A 'bad magic number' error usually means the binlog file '{binlog_file}' on the server is corrupted or has been rotated. You may need to run a new full backup.")
        else:
             print(f"Error during differential backup: {stderr_output}")
    except FileNotFoundError:
        print("Error: 'mysqlbinlog' command not found. Is MySQL client installed and in your system's PATH?")

differential_backup(
    user="root",
    password="1234",
    host="localhost",
    databases=["school_db"]  
)


## Restore full backup

In [ ]:
import subprocess

def restore_database(host, user, password, db_name, input_file):
    command = [
        "mysql",
        f"--host={host}",
        f"--user={user}",
        f"--password={password}",
        db_name
    ]
    print(command)
    
    with open(input_file, "r") as infile:
        result = subprocess.run(command, stdin=infile, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result)
    
    if result.returncode == 0:
        print(f"Restore successful from {input_file}.")
    else:
        print(f"Error: {result.stderr.decode('utf-8')}")

# Example usage
restore_database("localhost", "root", "1234", "computer_shop", "../backup/full/full_backup_2025-07-05_23-02-10.sql")


## Restore Full backup with differential backup

In [ ]:
import subprocess

def restore_database(host, user, password, db_name, input_fullbackup_file,input_diff_backup_file):
    command = [
        "mysql",
        f"--host={host}",
        f"--user={user}",
        f"--password={password}",
        db_name
    ]
    print(command)
    # restore fullbackup
    with open(input_fullbackup_file, "r") as infile:
        result = subprocess.run(command, stdin=infile, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result)
    
    if result.returncode == 0:
        print(f"Restore successful from {input_fullbackup_file}.")
    else:
        print(f"Error: {result.stderr.decode('utf-8')}")
    # restore last differential file
    with open(input_diff_backup_file, "r") as infile:
        result = subprocess.run(command, stdin=infile, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result)
    
    if result.returncode == 0:
        print(f"Restore successful from {input_diff_backup_file}.")
    else:
        print(f"Error: {result.stderr.decode('utf-8')}")

# Example usage
restore_database("localhost", "root", "1234", "school_db", 
                input_fullbackup_file="../backup/full/diff_backup_school_db_2025-07-12_19-41-15.sql",
                input_diff_backup_file="../backup/differential/diff_backup_school_db_2025-07-12_19-41-15.sql")
